# Creating pulses

This notebook will illustrate how to create different kinds of pulses for the beamline.

In [ ]:
import numpy as np
import scipp as sc
import tof

AA = sc.Unit('angstrom')
ms = sc.Unit('ms')

## The ESS pulse

Pulses are characterized by three aspects:

- the number of neutrons in the pulse
- the time at which each neutron is born inside the pulse
- the wavelength of each neutron inside the pulse

The default way of creating pulses is to choose a facility name and a number of neutrons.
Each facility defines time and wavelength probability distributions for the neutrons.

In [ ]:
pulse = tof.Pulse(facility='ess', neutrons=1_000_000)
pulse

To inspect the data in the pulse, we can either look at its `birth_times` or `wavelengths` properties

In [ ]:
pulse.wavelengths

or we can plot it with

In [ ]:
pulse.plot()

## Specifying time and wavelength distributions

It is also possible to create pulses with custom time and wavelength distributions.
For this, we need to use the `Pulse.from_distribution()` method.

### Flat distributions

By default, the time and wavelength distributions of neutrons are both flat (uniform).
We then need to define a pulse duration (start and end times) and a wavelength range.

To create a pulse with 1 million neutrons, uniformly distributed in the ranges of 1-3 ms for birth times,
and 1-10 Å for wavelengths, we write:

In [ ]:
pulse = tof.Pulse.from_distribution(
    neutrons=1_000_000,
    tmin=1.0 * ms,
    tmax=3.0 * ms,
    wmin=1.0 * AA,
    wmax=10.0 * AA,
)
pulse.plot()

### Custom distributions

Pulses at neutron facilities are rarely flat,
and it is thus useful to be able to supply custom distributions as arrays of probabilities.
The array values represent the probabilities, while the associated coordinates represent the values to be sampled from.

As an example, we create a triangular distribution for the neutron birth times,
and a linearly increasing distribution for the neutron wavelengths
(note that internally a linear interpolation is performed on the original data).

In [ ]:
v = np.arange(30.0)
p_time = sc.DataArray(
    data=sc.array(dims=['time'], values=np.concatenate([v, v[::-1]])),
    coords={'time': sc.linspace('time', 0.1, 6.0, len(v) * 2, unit='ms')},
)
p_wav = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[1.0, 4.0]),
    coords={
        'wavelength': sc.array(dims=['wavelength'], values=[1.0, 4.0], unit='angstrom')
    },
)

pulse = tof.Pulse.from_distribution(neutrons=200_000, p_time=p_time, p_wav=p_wav)
pulse.plot()

Note that the time and wavelength distributions are independent;
a neutron with a randomly selected birth time from `p_time` can adopt any wavelength in `p_wav`
(in other words, the two distributions are simply broadcast into a square 2D parameter space).

## Specifying neutrons manually

Finally, it is possible to simply specify a list of birth times and wavelengths manually to create a pulse via the `from_neutrons` method.

In [ ]:
pulse = tof.Pulse.from_neutrons(
    birth_times=sc.array(
        dims=['event'],
        values=[0.0, 0.1, 0.2, 0.56],
        unit='s',
    ),
    wavelengths=sc.array(dims=['event'], values=[5.0, 8.0, 11.0, 7.1], unit='angstrom'),
)
pulse.plot()